In [31]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import roc_auc_score


data = pd.read_csv('../Feature Engineering/data_post_pca.csv')

X_pca = data.drop('label', axis=1)
y = data['label']

# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the ANN model architecture
# model = Sequential()
# model.add(Dense(64, input_dim=X_train.shape[-1], activation='relu'))
# model.add(Dense(128, activation='relu'))
# # model.add(Dense(128, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[-1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))



# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))



# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")
y_prob = model.predict(X_test)
auc = roc_auc_score(y_test, y_prob)

print("AUC:", auc)


Epoch 1/20
480/480 [==============================] - 0s 709us/step - loss: 0.3607 - accuracy: 0.8384 - val_loss: 0.2872 - val_accuracy: 0.8886
Epoch 2/20
480/480 [==============================] - 0s 604us/step - loss: 0.2267 - accuracy: 0.9063 - val_loss: 0.1900 - val_accuracy: 0.9103
Epoch 3/20
480/480 [==============================] - 0s 592us/step - loss: 0.1716 - accuracy: 0.9159 - val_loss: 0.1640 - val_accuracy: 0.9134
Epoch 4/20
480/480 [==============================] - 0s 619us/step - loss: 0.1548 - accuracy: 0.9189 - val_loss: 0.1608 - val_accuracy: 0.9189
Epoch 5/20
480/480 [==============================] - 0s 623us/step - loss: 0.1427 - accuracy: 0.9252 - val_loss: 0.1446 - val_accuracy: 0.9163
Epoch 6/20
480/480 [==============================] - 0s 618us/step - loss: 0.1393 - accuracy: 0.9256 - val_loss: 0.1384 - val_accuracy: 0.9165
Epoch 7/20
480/480 [==============================] - 0s 603us/step - loss: 0.1282 - accuracy: 0.9266 - val_loss: 0.1279 - val_accuracy:

In [36]:
# Get the embedding of the last layer (penultimate layer)
embedding_layer_model = keras.Model(inputs=model.input, outputs=model.layers[-2].output)
embedding = embedding_layer_model.predict(X_train)

# Use the embedding to feed into KNN
k = 2
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(embedding, y_train)

480/480 [==============================] - 0s 259us/step


KNeighborsClassifier(n_neighbors=2)

In [37]:
# Getting accuracy score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

embedding_test = embedding_layer_model.predict(X_test)
# Use the trained classifier to make predictions on the test data
y_pred = knn.predict(embedding_test)

def evaluate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)
print("Accuracy:", evaluate_accuracy(y_test,y_pred))

150/150 [==============================] - 0s 315us/step
Accuracy: 0.9630634390651085


/Users/pratikjadhav/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
